# 2 : Dataset

In [7]:
from collections import Counter
import pandas as pd
import math
import re

# Truthful and deceptive training sets
with open('./DATASET/train/truthful.txt') as t, open('./DATASET/train/deceptive.txt') as d:
    T_TRAIN = t.read()
    D_TRAIN = d.read()
    
# Truthful and deceptive validation sets
with open('./DATASET/validation/truthful.txt') as t, open('./DATASET/validation/deceptive.txt') as d:
    T_VAL = t.read()
    D_VAL = d.read()

# Test set
with open('./DATASET/test/test.txt') as t:
    TEST = t.read()

# 3 : Unsmoothed N-Grams
### 3.1 : Preprocessing

In [631]:
def preprocess(text):
    """
    Returns a formatted list of the words in the input string [text]
    
    preprocess adds start characters at the beginning of each review in
    [text] and then splits the string on spaces
    
    text: the input string to be processed
    """
    return add_start_characters(text).split()
    
def add_start_characters(words):
    """
    Returns the modified string [words] with all newline [\n] characters replaced with start [<s>] characters
    
    words: a string
    """
    words = '<s> ' + words
    words = words.replace('\n', ' <s> ')
    return words[:-5] if words[-5:] == ' <s> ' else words

### 3.2 : Unsmoothed Unigram Probability

In [632]:
def get_unigram_corpus(wordlist):
    """
    Returns the unigram corpus given by the input [wordlist]
    
    get_unigram_corpus counts the number of occurrences of each unique token in [wordlist]
    and returns a dictionary where key:value pairings represent unigram:count
    
    wordlist: a list of words (strings)
    """
    return dict(Counter(wordlist))

def get_unigram_prob(unigram, unigram_corpus):
    """
    Returns the probability of a given [unigram] on a given [unigram_corpus]
    
    get_unigram_prob returns the ratio of the value of [unigram] in the dictionary [unigram_corpus] 
    (or 1 if it is not present) to the total sum of all values in the dictionary (where the values 
    are the counts of each unigram in the [unigram_corpus])
    
    unigram: a unigram (string)
    unigram_corpus: a dictionary of unigram:count pairings
    """
    acc = 0
    for key in unigram_corpus:
        acc += unigram_corpus[key]
    return unigram_corpus.get(unigram, 1) / acc

### 3.3 : Unsmoothed Bigram Probability

In [633]:
def get_bigram_corpus(wordlist):
    """
    Returns the bigram corpus given by the input [wordlist]
    
    get_bigram_corpus creates a dictionary with bigrams from the [wordlist]
    as keys and counts the instances of each bigram to assign values (except
    for the bigram ('.', '<s>') which represents the start of one review and
    the end of another)
    
    wordlist: a list of words (strings)
    """
    corpus = {}
    for i, word in enumerate(wordlist[1:], start=1):
        if word != '<s>':
            if (wordlist[i-1], word) not in corpus:
                corpus[(wordlist[i-1], word)] = 1
            else:
                corpus[(wordlist[i-1], word)] += 1
    return corpus

def get_bigram_prob(bigram, bigram_corpus):
    """
    Returns the probability of a given [bigram] on a given [bigram_corpus]
    
    get_bigram_corpus takes the ratio of the value of [bigram] in the dictionary
    [bigram_corpus] (or 1 if it is not present) to the sum of the values of all bigrams
    (keys) in the dictionary such that the they have the same first word as [bigram]
    
    bigram: a bigram (tuple of strings)
    bigram_corpus: a dictionary of bigram:count pairings
    """
    acc = 0
    for key in bigram_corpus:
        if key[0] == bigram[0]:
            acc += bigram_corpus[key]
    return bigram_corpus.get(bigram, 1) / acc

# 4 : Smoothing and Unknown Words
### 4.1 : Unknown Word Handling

In [634]:
def check_for_unk_words(wordlist, tokenlist):
    """
    Returns the [wordlist] filtered by the [tokenlist]
    
    check_for_unk_words takes any element of [wordlist] and replaces it with the
    unknown character string [<UNK>] if it does not exist in the [tokenlist]
    
    wordlist: a list of words (strings)
    tokenlist: a list of tokens (strings)
    """
    for i, token in enumerate(wordlist):
        if token not in tokenlist:
              wordlist[i] = '<UNK>'
    return wordlist

### 4.2 : Smooth Bigram Probability

In [635]:
def get_smooth_bigram_corpus(tokenlist, bigram_corpus):
    """
    Returns a dataframe object where the columns and rows are labeled with the tokens
    in [tokenlist] and the elements are the counts of each bigram (defaulting to 1 to handle
    add-one smoothing) in the format (row, column) so that the count for bigram (x, y) is 
    found with df.loc[x, by]
    
    get_smooth_bigram_corpus also appends the unknown word character [<UNK>] to handle 
    unknown words
    
    tokenlist: a list of tokens (strings)
    bigram_corpus: a dictionary of bigram:count pairings
    """
    tokenlist.append('<UNK>')
    df = pd.DataFrame(1, index = tokenlist, columns = tokenlist) 
    for bigram in bigram_corpus:
        df.loc[bigram[0], bigram[1]] += bigram_corpus[bigram]
    return df

def get_smooth_bigram_prob(bigram, smooth_bigram_corpus):
    """
    Returns the probability of a given [bigram] on a given [smooth_bigram_corpus]
    
    get_smooth_bigram_prob takes the ratio of the value of [bigram] (df.loc(bigram[0], bigram[1]))
    in the table [smooth_bigram_corpus] to the sum of all elements in the same row
    
    bigram: a bigram (tuple of strings)
    smooth_bigram_corpus: a dataframe with tokens as row and column names and bigram counts as values
    """
    return smooth_bigram_corpus.loc[bigram[0], bigram[1]] / smooth_bigram_corpus.loc[bigram[0]].sum()

# 5 : Perplexity

In [636]:
class NGramModel():
    """
    Basic format for n-gram language based model for opinion spam classification
    """
    def __init__(self, *args):
        super(NGramModel, self).__init__()

    "Calculate the perplexity of the model on a given corpus"
    def get_perp(self, *args):
        return



class UnigramModel(NGramModel):
    """
    Unigram language based model for opinion spam classification
    
    data: a preprocessed list of words (strings)
    """
    def __init__(self, data):
        super(UnigramModel, self).__init__()
        self.corpus = get_unigram_corpus(data)

    """
    Returns the perplexity of the unigram model on the [test_corpus]
    
    get_perp exponentiates the normalized sum of the log probabilities of each
    token (unigram) in the [test_corpus]
    
    test_corpus: a preprocessed corpus (list of strings)
    """
    def get_perp(self, test_corpus):
        N = len(test_corpus)
        acc = 0
        for word in test_corpus:
            acc -= math.log(get_unigram_prob(word, self.corpus))
        return math.exp((1/N) * acc)



class BigramModel(NGramModel):
    """
    Bigram language based model for opinion spam classification
    
    data: a preprocessed list of words (strings)
    """
    def __init__(self, data):
        super(BigramModel, self).__init__()
        self.tokens = list(get_unigram_corpus(data).keys())
        self.corpus = get_bigram_corpus(data)

    """
    Returns the perplexity of the bigram model on the [test_corpus]
    
    get_perp exponentiates the normalized sum of the log probabilities of each
    bigram in the [test_corpus]; for N tokens in the [test_corpus], we create N-1 
    bigrams and take the probabilities of those, eventually normalizing by dividing by N-1
    
    test_corpus: a preprocessed corpus (list of strings)
    """    
    def get_perp(self, test_corpus):
        N = len(test_corpus)
        acc = 0
        for i, word in enumerate(test_corpus):
            if i == 0:
                continue
            bigram = (test_corpus[i-1], word)
            acc -= math.log(get_bigram_prob(bigram, self.corpus))
        return math.exp(1/(N-1) * acc)



class SmoothBigramModel(NGramModel):
    """
    Smooth bigram language based model for opinion spam classification
    
    data: a preprocessed list of words (strings)
    """
    def __init__(self, data):
        super(SmoothBigramModel, self).__init__()
        self.tokens = list(get_unigram_corpus(data).keys())
        corpus = get_bigram_corpus(data)
        self.corpus = get_smooth_bigram_corpus(self.tokens, corpus)

    """
    Returns the perplexity of the smooth bigram model on the [test_corpus]
    
    get_perp exponentiates the normalized sum of the log probabilities of each
    bigram in the [test_corpus]; for N tokens in the [test_corpus], we create N-1 
    bigrams and take the probabilities of those, eventually normalizing by dividing by N-1
    
    test_corpus: a preprocessed corpus (list of strings)
    """
    def get_perp(self, test_corpus):
        N = len(test_corpus)
        acc = 0
        for i, word in enumerate(test_corpus):
            if i == 0:
                continue
            bigram = (test_corpus[i-1], word)
            acc -= math.log(get_smooth_bigram_prob(bigram, self.corpus))
        return math.exp(1/(N-1) * acc)

In [637]:
# MODELS: a unigram model and a smooth bigram model

truthful_unigram_model = UnigramModel(preprocess(T_TRAIN))
deceptive_unigram_model = UnigramModel(preprocess(D_TRAIN))

truthful_smooth_bigram_model = SmoothBigramModel(preprocess(T_TRAIN))
deceptive_smooth_bigram_model = SmoothBigramModel(preprocess(D_TRAIN))

In [638]:
# VALIDATION DATA

# Truthful validation reviews filtered on truthful training tokens
T_VAL_T = check_for_unk_words(preprocess(T_VAL), truthful_smooth_bigram_model.tokens)

# Deceptive validation reviews filtered on truthful training tokens
D_VAL_T = check_for_unk_words(preprocess(D_VAL), truthful_smooth_bigram_model.tokens)

# Truthful validation reviews filtered on truthful deceptive tokens
T_VAL_D = check_for_unk_words(preprocess(T_VAL), deceptive_smooth_bigram_model.tokens)

# Deceptive validation reviews filtered on truthful deceptive tokens
D_VAL_D = check_for_unk_words(preprocess(D_VAL), deceptive_smooth_bigram_model.tokens)

In [640]:
# PERPLEXITIES

print(truthful_unigram_model.get_perp(T_VAL_T))
print(truthful_unigram_model.get_perp(D_VAL_T))
print(deceptive_unigram_model.get_perp(T_VAL_D))
print(deceptive_unigram_model.get_perp(D_VAL_D))

print(truthful_smooth_bigram_model.get_perp(T_VAL_T))
print(truthful_smooth_bigram_model.get_perp(D_VAL_T))
print(deceptive_smooth_bigram_model.get_perp(T_VAL_D))
print(deceptive_smooth_bigram_model.get_perp(D_VAL_D))

575.8911106150354
507.2203959017036
615.8537298038397
463.8680901825986
1454.340989231253
1281.9503153726475
1324.6771814849794
958.4681090581158


In [643]:
# DATASET PROCESSING TOOLS

def separate_reviews(wordlist):
    """
    Returns the wordlist (which should correspond to a list of reviews) separated by 
    start [<s>] characters, so that each element of the return result is itself a list
    of the words of a single review
    
    wordlist: a list of words (strings)
    """
    start = 0
    reviews = []
    for i, word in enumerate(wordlist):
        if word == '<s>' and i != 0:
            reviews.append(wordlist[start:i])
            start = i+1
    return reviews

def separate_and_label_reviews(wordlist, label):
    """
    Returns a dictionary mapping the wordlist (which should correspond to a list of reviews) separated by 
    start [<s>] characters, so that each element of the return result is itself a list
    of the words of a single review
    
    wordlist: a list of words (strings)
    """
    sep_reviews = separate_reviews(wordlist)
    reviews = {}
    for review in sep_reviews:
        reviews[tuple(review)] = label
    return reviews

In [644]:
# TOKENS is an intersecting set of all tokens from the truthful training data and the deceptive training data
TOKENS = list(set(truthful_smooth_bigram_model.tokens) & set(deceptive_smooth_bigram_model.tokens))

# VAL_REVIEWS_T is a list of words from the truthful validation set filtered on TOKENS 
VAL_REVIEWS_T = check_for_unk_words(preprocess(T_VAL), TOKENS)

# VAL_REVIEWS_D is a list of words from the deceptive validation set filtered on TOKENS 
VAL_REVIEWS_D = check_for_unk_words(preprocess(D_VAL), TOKENS)

# VAL_REVIEWS is a dictionary where the keys are wordlists of the validation (truthful and deceptive) reviews
# and the values are the classification labels, with 0 representing truthful and 1 deceptive,
# and all possible unknown words have been filtered out
VAL_REVIEWS = {**separate_and_label_reviews(VAL_REVIEWS_T, 0), **separate_and_label_reviews(VAL_REVIEWS_D, 1)}

In [645]:
def classify(review, tmodel, dmodel):
    """
    Returns 0 if the models predict that the review is truthful, 1 otherwise
    
    classify compares the relative perplexities of truthfully-trained and deceptively-trained
    models on the review and chooses the classification based on which model computes a smaller
    perplexity on the corpus in question
    
    review: a preprocessed list of words (strings)
    tmodel: a language based model trained on truthful reviews
    dmodel: a language based model trained on deceptive reviews
    """
    return 0 if tmodel.get_perp(review) < dmodel.get_perp(review) else 1

def validate(reviews, tmodel, dmodel):
    """
    Returns the accuracy of a language based model on the [reviews]
    
    validate takes the ratio of the number of correct classification predictions
    of the model on the [reviews] and the total number of reviews to compute
    the accuracy of the model
    
    reviews: a dictionary mapping wordlists (reviews) to labels (0, 1)
    tmodel: a language based model trained on truthful reviews
    dmodel: a language based model trained on deceptive reviews
    """
    acc = 0
    for review in reviews:
        if classify(review, tmodel, dmodel) == reviews[review]:
            acc += 1
    return acc / len(reviews)

In [646]:
# Compute the accuracy of the smooth bigram model on the validation set
print(validate(VAL_REVIEWS, truthful_smooth_bigram_model, deceptive_smooth_bigram_model))

# Compute the accuracy of the unigram model on the validation set
print(validate(VAL_REVIEWS, truthful_unigram_model, deceptive_unigram_model))

0.8740157480314961

### 6.2 : Kaggle Submission

In [647]:
def test(reviews, tmodel, dmodel):
    """
    Returns a dataframe with two columns, Ids and Predictions, where
    Ids are the indices of the order of the test reviews, and Predictions
    are the predicted classifications of the model (0, 1)
    
    reviews: a dictionary mapping wordlists (reviews) to labels (0, 1)
    tmodel: a language based model trained on truthful reviews
    dmodel: a language based model trained on deceptive reviews
    """
    ids = [i for i in range(len(reviews))]
    preds = []
    for review in reviews:
        preds.append(classify(review, tmodel, dmodel))
    df = pd.DataFrame({'Id': ids, 'Prediction': preds}, columns = ['Id', 'Prediction'])
    return df

In [650]:
# TEST_REVIEWS is a list of words from the test set filtered on TOKENS
# and then partitioned into individual reviews
TEST_REVIEWS = check_for_unk_words(preprocess(TEST), TOKENS)
TEST_REVIEWS = separate_reviews(TEST_REVIEWS)
TEST_REVIEWS.append([])

# df is a dataframe of the smooth bigram model's predictions for the reviews in the test set
df = test(TEST_REVIEWS, truthful_smooth_bigram_model, deceptive_smooth_bigram_model)
df.to_csv('Smooth-Bigram-LM.csv')
df

,Id,Prediction
0,0,0
1,1,1
2,2,1
3,3,1
4,4,1
...,...,...
315,315,1
316,316,1
317,317,1
318,318,1


In [ ]:
# Toy example for verifying correctness of smooth bigram probabilities

# toy_train = preprocess('I like to run and I hate to have to do homework')
# toy_model = SmoothBigramModel(toy_train)
# print(toy_model.corpus)
# toy_val = check_for_unk_words(preprocess('I like homework'), toy_model.tokens)
# print(toy_model.get_perp(toy_val))

### 6.3 : Feature-based Naive Bayes for Opinion Spam Classification

#### Model 1: Unigram/Bag of Words Naive Bayes Classifier

In [16]:
class bag_of_words_nb_classifier():
    def __init__(self, training_truthful_text, training_deceptive_text):
        self.truthful_text = self.add_start_characters(training_truthful_text)
        self.deceptive_text = self.add_start_characters(training_deceptive_text)
        
        self.truthful_reviews_Counter = Counter(self.truthful_text.split())
        self.deceptive_reviews_Counter = Counter(self.deceptive_text.split())
        
        self.truthful_total_words = sum(self.truthful_reviews_Counter.values())
        self.deceptive_total_words = sum(self.deceptive_reviews_Counter.values())
        
        self.truthful_reviews_len = len(self.truthful_text.split('\n'))
        self.deceptive_reviews_len = len(self.truthful_text.split('\n'))

        self.both_reviews_Counter = self.truthful_reviews_Counter + self.deceptive_reviews_Counter
        self.vocabulary_size = len(self.both_reviews_Counter.keys())
        self.word_pattern = re.compile("(\w+|<s> |[,.!;])")
        
        self.k = 0.2

    def add_start_characters(self, words):
        words = '<s> ' + words
        words = words.replace('\n', ' <s> ')
        return words[:-5]
        
    def smoothed_word_log_prob(self, word, counter, total):
        return math.log((counter[word] + self.k) / (total + (self.vocabulary_size*self.k)))
    
    
    def smoothed_review_log_prob(self, review, counter, total):
        log_prob = 0.0
        for word in self.word_pattern.findall(review):
            log_prob += self.smoothed_word_log_prob(word, counter, total)
        return log_prob

    
    def classify_review(self, review):
        review = '<s> ' + review
        truthful_prob = self.smoothed_review_log_prob(review,
                            self.truthful_reviews_Counter, self.truthful_total_words)
        deceptive_prob = self.smoothed_review_log_prob(review,
                                self.deceptive_reviews_Counter, self.deceptive_total_words)

        # get ratio between the two (since this training set has the same number of reviews, this code is optional)
        truthful_prob = truthful_prob + \
            math.log(self.truthful_reviews_len/(self.truthful_reviews_len + self.deceptive_reviews_len))
        deceptive_prob = deceptive_prob + \
            math.log(self.deceptive_reviews_len /(self.truthful_reviews_len + self.deceptive_reviews_len))

        return 0 if truthful_prob >= deceptive_prob else 1

#### Training

In [17]:
with open('./DATASET/train/truthful.txt') as t, open('./DATASET/train/deceptive.txt') as d:
    truthful = t.read()
    deceptive = d.read()
nb = bag_of_words_nb_classifier(truthful, deceptive)

#### Validation

In [18]:
with open('./DATASET/validation/truthful.txt') as t, open('./DATASET/validation/deceptive.txt') as d:
    truthful_validation_text = t.read()
    deceptive_validation_text = d.read()

In [21]:
truthful_validation_classifications = \
    [nb.classify_review(review) for review in truthful_validation_text.split('\n')]
truthful_validation_accuracy_counts = Counter(truthful_validation_classifications)
print(truthful_validation_accuracy_counts)
print('Accuracy rate:', 
      truthful_validation_accuracy_counts[0]/sum(truthful_validation_accuracy_counts.values()))

Counter({0: 120, 1: 9})
Accuracy rate: 0.9302325581395349


In [22]:
deceptive_validation_classifications = \
    [nb.classify_review(review) for review in deceptive_validation_text.split('\n')]
deceptive_validation_accuracy_counts = Counter(deceptive_validation_classifications)
print(deceptive_validation_accuracy_counts)
print('Accuracy rate:', 
      deceptive_validation_accuracy_counts[1]/sum(deceptive_validation_accuracy_counts.values()))

Counter({1: 121, 0: 8})
Accuracy rate: 0.937984496124031


#### Testing

In [23]:
with open('./DATASET/test/test.txt') as t:
    test_text = t.read()

    deceptive_validation_classifications = \
    test_results=[nb.classify_review(review) for review in test_text.split('\n')]

In [24]:
test_results = test_results[:-1]
test_ids = [id_ for id_ in range(0, len(test_text.split('\n')) - 1)]

In [29]:
df = pd.DataFrame({'Id' : test_ids,'Prediction':test_results}, columns=['Id', 'Prediction'])
df.head(10)

,Id,Prediction
0,0,0
1,1,0
2,2,1
3,3,1
4,4,0
5,5,0
6,6,1
7,7,1
8,8,1
9,9,0


In [30]:
df.to_csv('naive_bayes_unigram.csv')

#### Model 2: Bigram Naive Bayes Classifier
* Built on top of the NLTK Library

In [31]:
import nltk.classify.util
from nltk.classify import NaiveBayesClassifier
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk import ngrams

In [44]:
class bigram_nb_classifier():
    '''
    A bigram naive bayes classifier built on top of the NLTK library.
    
    '''
    
    def __init__(self, truthful_train_txt, deceptive_train_txt):
        '''
        Initialize with the raw text of the reviews from the text files, delimited by newlines.
        
        truthful_train_txt: One string containing all truthful reviews to be trained, delimited by newlines.
        deceptive_train_txt: One string all truthful reviews to be trained, delimited by newlines.
        '''
        truthful_data = [(self.create_ngram_features(review.split()), 'truthful') \
                         for review in self.preprocess(truthful_train_txt)]
        deceptive_data = [(self.create_ngram_features(review.split()), 'deceptive') \
                          for review in self.preprocess(deceptive_train_txt)]
        self.training_data = deceptive_data + truthful_data
        self.classifier = NaiveBayesClassifier.train(self.training_data)
        self.latest_accuracy = -1
        
        stoplist = set(stopwords.words("english"))
    
    # Method from 
    # https://stackoverflow.com/questions/48003907/how-to-train-naive-bayes-classifier-for-n-gram-movie-reviews
    def create_ngram_features(self, words, n=2):
        '''
        Will create a dictionary of bigrams in the form {(word1, word2), True} for input into the NLTK 
        classifier. 
        
        words: words of 1 review to be converted into Bigrams.
        n: n value in n-grams.
        '''
        ngram_vocab = ngrams(words, n)
        my_dict = dict([(ng, True) for ng in ngram_vocab])
        return my_dict
    
    def compute_accuracy(self, truthful_val_txt, deceptive_val_txt):
        '''
        Computes the accuracy against the validation set.
        
        truthful_val_txt: One string containing all truthful reviews to be validated, 
            delimited by newlines.
        deceptive_val_txt: One string containing all deceptive reviews to be validated, 
            delimited by newlines.
        '''
        truthful_data_v = [(self.create_ngram_features(review.split()), 'truthful') \
                           for review in self.preprocess(truthful_val_txt)]
        deceptive_data_v = [(self.create_ngram_features(review.split()), 'deceptive') \
                            for review in self.preprocess(deceptive_val_txt)]
        validation_data = truthful_data_v + deceptive_data_v 
        self.latest_accuracy = nltk.classify.util.accuracy(self.classifier, validation_data)
        return self.latest_accuracy
    
    def add_start_character(self, review):
        '''
        Adds a start token (<s>) to the beginning of the string.
        
        review: Review to add the token to the start of.
        '''
        return '<s> ' + review

    def preprocess(self, text):
        '''
        Splits reviews from the review list into individual reviews and adds a start character to 
        each one. 
        
        text: Raw text from training text as a string.
        '''
        review_list = text.split('\n')[:-1]
        processed_review_list = [self.add_start_character(review) for review in review_list]
        return processed_review_list
    
    def classify_review(self, review):
        '''
        Classifies a review as either truthful or deceptive.
        
        review: Review as a string to be classified.
        '''
        review = self.add_start_character(review)
        return self.classifier.classify(self.create_ngram_features(review.split()))

#### Training

In [45]:
with open('./DATASET/train/truthful.txt') as t, open('./DATASET/train/deceptive.txt') as d:
    truthful_train_txt = t.read()
    deceptive_train_txt = d.read()

In [46]:
bnb = bigram_nb_classifier(truthful_train_txt, deceptive_train_txt)

#### Validation

In [47]:
with open('./DATASET/validation/truthful.txt') as t, open('./DATASET/validation/deceptive.txt') as d:
    truthful_val_txt = t.read()
    deceptive_val_txt = d.read()

In [48]:
bnb.compute_accuracy(truthful_val_txt, deceptive_train_txt)

0.9828125

#### Testing

In [49]:
with open('./DATASET/test/test.txt') as t:
    test_txt = t.read()

In [50]:
for review in test_txt.split('\n')[:-1]:
    print(bnb.classify_review(review))

truthful
truthful
deceptive
deceptive
truthful
truthful
deceptive
deceptive
deceptive
truthful
deceptive
deceptive
deceptive
truthful
deceptive
deceptive
deceptive
truthful
deceptive
truthful
truthful
deceptive
deceptive
deceptive
deceptive
deceptive
deceptive
truthful
deceptive
truthful
deceptive
truthful
truthful
truthful
deceptive
truthful
deceptive
truthful
deceptive
deceptive
deceptive
deceptive
deceptive
deceptive
deceptive
deceptive
truthful
deceptive
deceptive
deceptive
truthful
deceptive
truthful
deceptive
deceptive
deceptive
truthful
deceptive
truthful
deceptive
deceptive
deceptive
truthful
deceptive
truthful
deceptive
truthful
truthful
truthful
deceptive
deceptive
truthful
truthful
truthful
truthful
deceptive
deceptive
deceptive
deceptive
deceptive
truthful
truthful
truthful
truthful
truthful
truthful
truthful
truthful
truthful
truthful
deceptive
truthful
truthful
truthful
deceptive
deceptive
deceptive
deceptive
truthful
truthful
truthful
truthful
truthful
deceptive
truthful